<a href="https://colab.research.google.com/github/bzhao10/cs598_dlh_final_project/blob/main/model_without_cat_rolled_up.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

CS 598 Final Project

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
notes=pd.read_csv("/content/drive/MyDrive/DLH_final_project_dataset/preprocessed_rolled_up_icd_codes_10000.csv")

print(notes)

      Unnamed: 0                                               TEXT  \
0             41  ccu nursing progress notepls see transfer summ...   
1             42  nursing progress notes month only i have some ...   
2             43  respiratory care pt extubated to a aerosol avr...   
3             44  respiratory care patient remains on cpap psv v...   
4             45  repeat k hct due post blood transfusion lactol...   
...          ...                                                ...   
9995       10888  title case management initial assessment copy ...   
9996       13175  discharge plan this nurse case manager was not...   
9997       16260  case management discharge planning note this n...   
9998        4889  title cardiology follow up events patient note...   
9999        3652  pharmacy note digibind dose assessment pharmac...   

      rolled_up_code_008  rolled_up_code_009  rolled_up_code_010  \
0                      0                   0                   0   
1          

In [3]:
rolled_up_codes_list = notes.loc[:, notes.columns.str.startswith('rolled_up_code_')].columns.tolist()
print(rolled_up_codes_list)

['rolled_up_code_008', 'rolled_up_code_009', 'rolled_up_code_010', 'rolled_up_code_038', 'rolled_up_code_041', 'rolled_up_code_042', 'rolled_up_code_053', 'rolled_up_code_070', 'rolled_up_code_078', 'rolled_up_code_079', 'rolled_up_code_112', 'rolled_up_code_117', 'rolled_up_code_135', 'rolled_up_code_154', 'rolled_up_code_155', 'rolled_up_code_156', 'rolled_up_code_157', 'rolled_up_code_162', 'rolled_up_code_171', 'rolled_up_code_172', 'rolled_up_code_174', 'rolled_up_code_193', 'rolled_up_code_197', 'rolled_up_code_198', 'rolled_up_code_201', 'rolled_up_code_202', 'rolled_up_code_203', 'rolled_up_code_204', 'rolled_up_code_205', 'rolled_up_code_208', 'rolled_up_code_209', 'rolled_up_code_211', 'rolled_up_code_214', 'rolled_up_code_215', 'rolled_up_code_218', 'rolled_up_code_225', 'rolled_up_code_227', 'rolled_up_code_228', 'rolled_up_code_238', 'rolled_up_code_241', 'rolled_up_code_242', 'rolled_up_code_244', 'rolled_up_code_250', 'rolled_up_code_252', 'rolled_up_code_253', 'rolled_u

In [4]:
import re
import nltk
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize
from nltk import word_tokenize
nltk.download('punkt')
class Preprocessing:
  def __init__(self, num_words, seq_len, target, binary=True):
    self.data = '/content/drive/MyDrive/DLH_final_project_dataset/preprocessed_rolled_up_icd_codes_10000.csv'
    self.num_words = num_words
    self.seq_len = seq_len
    self.vocabulary = None
    self.x_tokenized = None
    self.x_padded = None
    self.x_raw = None
    self.y = None
    
    self.x_train = None
    self.x_test = None
    self.y_train = None
    self.y_test = None

    self.target = target
    
  def load_data(self, binary):
    # Reads the raw csv file and split into
    # sentences (x) and target (y)
    df = pd.read_csv(self.data)
    self.x_raw = df['TEXT'].values
    if (binary==False):
      #df['rolled_up_codes'] = df.loc[:, rolled_up_codes_list[0]:rolled_up_codes_list[len(rolled_up_codes_list)-1]].values.tolist()
      df['rolled_up_codes'] = df.loc[:, 'rolled_up_code_008':'rolled_up_code_V88'].values.tolist()
      self.y = df['rolled_up_codes'].values
    else:
      self.y = df[self.target].values
    
  def text_tokenization(self):
    # Tokenizes each sentence by implementing the nltk tool
    self.x_raw = [word_tokenize(x) for x in self.x_raw]
    
  def build_vocabulary(self):
    # Builds the vocabulary and keeps the "x" most frequent word
    self.vocabulary = dict()
    fdist = nltk.FreqDist()
    
    for sentence in self.x_raw:
      for word in sentence:
        fdist[word] += 1
        
    common_words = fdist.most_common(self.num_words)
    
    for idx, word in enumerate(common_words):
      self.vocabulary[word[0]] = (idx+1)
  
  def word_to_idx(self):
    # By using the dictionary (vocabulary), it is transformed
    # each token into its index based representatio
    self.x_tokenized = list() 
    
    for sentence in self.x_raw:
      temp_sentence = list()
      for word in sentence:
        if word in self.vocabulary.keys():
          temp_sentence.append(self.vocabulary[word])
      self.x_tokenized.append(temp_sentence)
      
  def padding_sentences(self):
    # Each sentence which does not fulfill the required le
    # it's padded with the index 0
    pad_idx = 0
    self.x_padded = list()
    
    for sentence in self.x_tokenized:
      while len(sentence) < self.seq_len:
        sentence.insert(len(sentence), pad_idx)
      self.x_padded.append(sentence)
      
    self.x_padded = np.array(self.x_padded)
    
  def split_data(self):
    self.x_train, self.x_test, self.y_train, self.y_test = train_test_split(self.x_padded, self.y, test_size=0.25, random_state=42)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [5]:
#Bag of Trick's model for binary prediction
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
class BoT(nn.ModuleList):

   def __init__(self, seq_len=2200,num_words=45173,embedding_size=64,out_size=32):
      super(BoT, self).__init__()

      # Parameters regarding text preprocessing
      self.seq_len = seq_len
      self.num_words = num_words
      self.embedding_size = embedding_size
  
      # Kernel sizes
      self.kernel_1 = 3
   
      # Output size for each convolution
      self.out_size = out_size
      
      # Embedding layer definition
      self.embedding = nn.Embedding(self.num_words + 1, self.embedding_size, padding_idx=0)
      
      # Fully connected layer definition
      self.fc = nn.Linear(64, 1)

   def forward(self, x):

      x = self.embedding(x)
      #x = [batch size, seq. length, hidden dim]
      
      x = F.avg_pool2d(x, (x.shape[1], 1)).squeeze(1) #averages word vectors across whole sequence length
      #x = [batch size, hidden dim]

      x = self.fc(x)
      #x = [batch size, output dim]

      # Activation function is applied
      x = torch.sigmoid(x)
      
      return x.squeeze()

In [6]:
#Bag of Trick's model for multi-label prediction
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
class BoT_mul(nn.ModuleList):

   def __init__(self, seq_len=2200,num_words=45173, labels=362, embedding_size=64,out_size=32):
      super(BoT_mul, self).__init__()

      # Parameters regarding text preprocessing
      self.seq_len = seq_len
      self.num_words = num_words
      self.embedding_size = embedding_size
  
      # Kernel sizes
      self.kernel_1 = 3
   
      # Output size for each convolution
      self.out_size = out_size
      
      # Embedding layer definition
      self.embedding = nn.Embedding(self.num_words + 1, self.embedding_size, padding_idx=0)
      
      # Fully connected layer definition
      self.fc = nn.Linear(64, labels)

   def forward(self, x):

      x = self.embedding(x)
      #x = [batch size, seq. length, hidden dim]
      
      x = F.avg_pool2d(x, (x.shape[1], 1)).squeeze(1) #averages word vectors across whole sequence length
      #x = [batch size, hidden dim]

      x = self.fc(x)
      #x = [batch size, output dim]

      # Activation function is applied
      x = torch.sigmoid(x)
      
      return x.squeeze()

In [7]:
# CNN baseline model for binary prediction
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
class CNN_baseline(nn.ModuleList):

   def __init__(self, seq_len=2200,num_words=45173,embedding_size=64,out_size=32):
      super(CNN_baseline, self).__init__()

      # Parameters regarding text preprocessing
      self.seq_len = seq_len
      self.num_words = num_words
      self.embedding_size = embedding_size
      
      # CNN parameters definition
      # Kernel sizes
      self.kernel_1 = 3

  
      # Output size for each convolution
      self.out_size = out_size
      
      # Embedding layer definition
      self.embedding = nn.Embedding(self.num_words + 1, self.embedding_size, padding_idx=0)
      
      # Convolution layers definition
      self.conv_1 = nn.Conv1d(self.seq_len, self.out_size, self.kernel_1)
      
      # Max pooling layers definition
      self.pool_1 = nn.MaxPool1d(self.kernel_1)
      
      # Fully connected layer definition
      self.fc = nn.Linear(640, 1)
 
   def forward(self, x):
      
      x = self.embedding(x)
      #x = [batch size, seq. length, hidden dim]

      # Convolution layer 1 is applied
      x1 = self.conv_1(x)
      x1 = torch.relu(x1)
      x1 = self.pool_1(x1)
      
      # The output of each convolutional layer is concatenated into a unique vector
      x1=torch.flatten(x1, 1)
      # The "flattened" vector is passed through a fully connected layer
      out = self.fc(x1)
      # Activation function is applied
      out = torch.sigmoid(out)
      
      return out.squeeze()

In [8]:
class CNN_baseline_mul(nn.ModuleList):

   def __init__(self, seq_len=2200,num_words=45173,labels=362,embedding_size=64,out_size=32):
      super(CNN_baseline_mul, self).__init__()

      # Parameters regarding text preprocessing
      self.seq_len = seq_len
      self.num_words = num_words
      self.embedding_size = embedding_size
      
      # CNN parameters definition
      # Kernel sizes
      self.kernel_1 = 3

      
      # Output size for each convolution
      self.out_size = out_size
      
      # Embedding layer definition
      self.embedding = nn.Embedding(self.num_words + 1, self.embedding_size, padding_idx=0)
      
      # Convolution layers definition
      self.conv_1 = nn.Conv1d(self.seq_len, self.out_size, self.kernel_1)
      
      # Max pooling layers definition
      self.pool_1 = nn.MaxPool1d(self.kernel_1)
      
      # Fully connected layer definition
      self.fc = nn.Linear(640, labels)
 
   def forward(self, x):
      
      # Sequence of tokes is filterd through an embedding layer
      x = self.embedding(x)

      # Convolution layer 1 is applied
      x1 = self.conv_1(x)
      x1 = torch.relu(x1)
      x1 = self.pool_1(x1)
      
      # The output of each convolutional layer is concatenated into a unique vector
      x1=torch.flatten(x1, 1)
      # The "flattened" vector is passed through a fully connected layer
      out = self.fc(x1)
      # Activation function is applied
      out = torch.sigmoid(out)
      
      return out.squeeze()

In [9]:

import math
import torch
import torch.nn as nn
import torch.nn.functional as F

class CNN_Three_Conv_multi(nn.ModuleList):

  def __init__(self,seq_len=2200,num_words=45173, labels= 362, embedding_size=64,out_size=32,stride=2):
          super(CNN_Three_Conv_multi, self).__init__()

          # Parameters regarding text preprocessing
          self.seq_len = seq_len
          self.num_words = num_words
          self.embedding_size = embedding_size
          
          # Dropout definition
          self.dropout = nn.Dropout(0.33)
          
          # CNN parameters definition
          # Kernel sizes
          self.kernel_1 = 2
          self.kernel_2 = 3
          self.kernel_3 = 4
          
          
          # Output size for each convolution
          self.out_size = out_size
          # Number of strides for each convolution
          self.stride = stride
          
          # Embedding layer definition
          self.embedding = nn.Embedding(self.num_words + 1, self.embedding_size, padding_idx=0)
          
          # Convolution layers definition
          self.conv_1 = nn.Conv1d(self.seq_len, self.out_size, self.kernel_1, self.stride)
          self.conv_2 = nn.Conv1d(self.seq_len, self.out_size, self.kernel_2, self.stride)
          self.conv_3 = nn.Conv1d(self.seq_len, self.out_size, self.kernel_3, self.stride)

          
          # Max pooling layers definition
          self.pool_1 = nn.MaxPool1d(self.kernel_1, self.stride)
          self.pool_2 = nn.MaxPool1d(self.kernel_2, self.stride)
          self.pool_3 = nn.MaxPool1d(self.kernel_3, self.stride)
          
          # Fully connected layer definition
          self.fc = nn.Linear(self.in_features_fc(), labels)

          
  def in_features_fc(self):
          '''Calculates the number of output features after Convolution + Max pooling
                  
          Convolved_Features = ((embedding_size + (2 * padding) - dilation * (kernel - 1) - 1) / stride) + 1
          Pooled_Features = ((embedding_size + (2 * padding) - dilation * (kernel - 1) - 1) / stride) + 1
          
          source: https://pytorch.org/docs/stable/generated/torch.nn.Conv1d.html
          '''
          # Calcualte size of convolved/pooled features for convolution_1/max_pooling_1 features
          out_conv_1 = ((self.embedding_size - 1 * (self.kernel_1 - 1) - 1) / self.stride) + 1
          out_conv_1 = math.floor(out_conv_1)
          out_pool_1 = ((out_conv_1 - 1 * (self.kernel_1 - 1) - 1) / self.stride) + 1
          out_pool_1 = math.floor(out_pool_1)
          
          # Calcualte size of convolved/pooled features for convolution_2/max_pooling_2 features
          out_conv_2 = ((self.embedding_size - 1 * (self.kernel_2 - 1) - 1) / self.stride) + 1
          out_conv_2 = math.floor(out_conv_2)
          out_pool_2 = ((out_conv_2 - 1 * (self.kernel_2 - 1) - 1) / self.stride) + 1
          out_pool_2 = math.floor(out_pool_2)
          
          # Calcualte size of convolved/pooled features for convolution_3/max_pooling_3 features
          out_conv_3 = ((self.embedding_size - 1 * (self.kernel_3 - 1) - 1) / self.stride) + 1
          out_conv_3 = math.floor(out_conv_3)
          out_pool_3 = ((out_conv_3 - 1 * (self.kernel_3 - 1) - 1) / self.stride) + 1
          out_pool_3 = math.floor(out_pool_3)
          
          # Returns "flattened" vector (input for fully connected layer)
          return (out_pool_1 + out_pool_2 + out_pool_3) * self.out_size      
          
  def forward(self, x):

          # Sequence of tokes is filterd through an embedding layer
          x = self.embedding(x)
          
          # Convolution layer 1 is applied
          x1 = self.conv_1(x)
          x1 = torch.relu(x1)
          x1 = self.pool_1(x1)
          
          # Convolution layer 2 is applied
          x2 = self.conv_2(x)
          x2 = torch.relu((x2))
          x2 = self.pool_2(x2)

          # Convolution layer 3 is applied
          x3 = self.conv_3(x)
          x3 = torch.relu(x3)
          x3 = self.pool_3(x3)
          
          
          # The output of each convolutional layer is concatenated into a unique vector
          union = torch.cat((x1, x2, x3), 2)
          union = union.reshape(union.size(0), -1)

          # The "flattened" vector is passed through a fully connected layer
          out = self.fc(union)
          # Dropout is applied        
          out = self.dropout(out)
          # Activation function is applied
          out = torch.sigmoid(out)
          
          return out.squeeze()

In [10]:
import torch
import torch.optim as optim
import torch.nn.functional as F
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
from torch.utils.data import Dataset, DataLoader

class DatasetMaper(Dataset):

	def __init__(self, x, y):
		self.x = x
		self.y = y
		
	def __len__(self):
		return len(self.x)
		
	def __getitem__(self, idx):
		return self.x[idx], self.y[idx]

In [14]:
class Run:
    '''Training, evaluation and metrics calculation'''

    @staticmethod
    def train(model, data, multi=362, binary=True):
        
        # Initialize dataset maper
        train = DatasetMaper(data['x_train'], data['y_train'])
        test = DatasetMaper(data['x_test'], data['y_test'])
        
        # Initialize loaders
        loader_train = DataLoader(train, batch_size=32)
        loader_test = DataLoader(test, batch_size=32)
        
        # Define optimizer
        optimizer = optim.Adam(model.parameters(),lr=0.001)
        
        # Starts training phase
        for epoch in range(15):
            # Set model in training model
            model.train()
            predictions = []
            # Starts batch training
            for x_batch, y_batch in loader_train:
                if (binary==False):
                  y_batch = torch.stack(y_batch).T
                y_batch = y_batch.type(torch.FloatTensor)
            
                # Feed the model
                y_pred = model(x_batch)
                
                # Loss calculation
                loss = F.binary_cross_entropy(y_pred, y_batch)
                
                # Clean gradientes
                optimizer.zero_grad()
                
                # Gradients calculation
                loss.backward()
                
                # Gradients update
                optimizer.step()
                
                # Save predictions
                predictions += list(y_pred.detach().numpy())
            
            # Evaluation phase
            test_predictions = Run.evaluation(model, loader_test)
            if (binary==True):
              predictions=[1 if pred>0.5 else 0 for pred in predictions]

              test_predictions=[1 if pred>0.5 else 0 for pred in test_predictions]

              train_precision,train_recall,train_f1,_ = precision_recall_fscore_support(predictions,data['y_train'],average='binary')
              test_precision,test_recall,test_f1,_ = precision_recall_fscore_support(test_predictions,data['y_test'],average='binary')
              test_accuracy=accuracy_score(data['y_test'],test_predictions)
              print("Epoch: %d, loss: %.5f, Train precision: %.5f,Train recall: %.5f,Train f1: %.5f, Test accuracy: %.5f,Test precision: %.5f, Test recall: %.5f,Test f1: %.5f" % (epoch+1, loss.item(), train_precision,train_recall,train_f1,test_accuracy,test_precision,test_recall,test_f1))
            else:
              for i in range(len(predictions)):
                predictions[i]=[1 if pred>0.5 else 0 for pred in predictions[i]]  

              for i in range(len(test_predictions)):
                test_predictions[i]=[1 if pred>0.5 else 0 for pred in test_predictions[i]] 
              
              multi_label_predictions = np.array(predictions)
              y_train=np.empty([7500,multi])
              for i in range(len(data['y_train'])):
                y_train[i][:]=data['y_train'][i]

              train_precision_all = np.empty([multi])
              train_recall_all = np.empty([multi])
              train_f1_all = np.empty([multi])
              for i in range(len(predictions[0])):
                  train_precision_all[i],train_recall_all[i],train_f1_all[i],_ = precision_recall_fscore_support(multi_label_predictions[:][i],y_train[:][i],average='binary')
              train_precision = train_precision_all.mean()
              train_recall = train_recall_all.mean()
              train_f1 = train_f1_all.mean()

              multi_label_test_predictions = np.array(test_predictions)
              y_test=np.empty([2500,multi])
              for i in range(len(data['y_test'])):
                y_test[i][:]=data['y_test'][i]

              test_precision_all = np.empty([multi])
              test_recall_all = np.empty([multi])
              test_f1_all = np.empty([multi])
              for i in range(len(test_predictions[0])):
                  test_precision_all[i],test_recall_all[i],test_f1_all[i],_ = precision_recall_fscore_support(multi_label_test_predictions[:][i],y_test[:][i],average='binary')
              test_precision = test_precision_all.mean()
              test_recall = test_recall_all.mean()
              test_f1 = test_f1_all.mean()
              print("Epoch: %d, loss: %.5f, Train precision: %.5f,Train recall: %.5f,Train f1: %.5f, Test precision: %.5f, Test recall: %.5f,Test f1: %.5f" % (epoch+1, loss.item(), train_precision,train_recall,train_f1,test_precision,test_recall,test_f1))
            
    @staticmethod
    def evaluation(model, loader_test):
        
        # Set the model in evaluation mode
        model.eval()
        predictions = []
        
        # Starst evaluation phase
        with torch.no_grad():
            for x_batch, y_batch in loader_test:
                y_pred = model(x_batch)
                predictions += list(y_pred.detach().numpy())
        return predictions
        
    @staticmethod
    def calculate_accuray(grand_truth, predictions):
        # Metrics calculation
        true_positives = 0
        true_negatives = 0 
        for true, pred in zip(grand_truth, predictions):
            if (pred >= 0.5) and (true == 1):
                true_positives += 1
            elif (pred < 0.5) and (true == 0):
                true_negatives += 1
            else:
                pass
        # Return accuracy
        return (true_positives+true_negatives) / len(grand_truth)

In [15]:
def prepare_data(num_words, seq_len, target, binary=True):
  # Preprocessing pipeline
  pr = Preprocessing(num_words, seq_len, target, binary)
  pr.load_data(binary)
  pr.text_tokenization()
  pr.build_vocabulary()
  pr.word_to_idx()
  pr.padding_sentences()
  pr.split_data()

  return {'x_train': pr.x_train, 'y_train': pr.y_train, 'x_test': pr.x_test, 'y_test': pr.y_test}

In [22]:
binary_data=prepare_data(45173, 2200, 'rolled_up_code_584', binary=True)

In [ ]:
multi_label_data = prepare_data(45173, 2200, 'rolled_up_code_518', binary=False)

In [ ]:
model_BoT=BoT()
Run().train(model_BoT, binary_data, len(rolled_up_codes_list), binary=True)

Epoch: 1, loss: 0.68180, Train precision: 0.16633,Train recall: 0.72671,Train f1: 0.27071, Test accuracy: 0.60960,Test precision: 0.36878, Test recall: 0.65714,Test f1: 0.47243
Epoch: 2, loss: 0.63790, Train precision: 0.52488,Train recall: 0.69061,Train f1: 0.59645, Test accuracy: 0.66400,Test precision: 0.51814, Test recall: 0.69536,Test f1: 0.59381
Epoch: 3, loss: 0.59362, Train precision: 0.57066,Train recall: 0.76956,Train f1: 0.65535, Test accuracy: 0.71520,Test precision: 0.52068, Test recall: 0.81078,Test f1: 0.63412
Epoch: 4, loss: 0.53793, Train precision: 0.57976,Train recall: 0.85350,Train f1: 0.69048, Test accuracy: 0.73520,Test precision: 0.53502, Test recall: 0.85101,Test f1: 0.65699
Epoch: 5, loss: 0.48162, Train precision: 0.59710,Train recall: 0.87500,Train f1: 0.70982, Test accuracy: 0.74280,Test precision: 0.55274, Test recall: 0.85286,Test f1: 0.67076
Epoch: 6, loss: 0.43273, Train precision: 0.61530,Train recall: 0.87825,Train f1: 0.72362, Test accuracy: 0.75200,T

In [ ]:
model_BoT_mul=BoT_mul()
print(len(rolled_up_codes_list))
Run().train(model_BoT_mul, multi_label_data, len(rolled_up_codes_list), binary=False)

In [ ]:
model_CNN_baseline=CNN_baseline()
Run().train(model_CNN_baseline, binary_data, len(rolled_up_codes_list), binary=True)

In [ ]:
model_CNN_baseline_mul=CNN_baseline_mul()
Run().train(model_CNN_baseline_mul, multi_label_data, len(rolled_up_codes_list), binary=False)

In [ ]:
model_CNN_Three_Conv_multi = CNN_Three_Conv_multi()
Run().train(model_CNN_Three_Conv_multi, multi_label_data, len(rolled_up_codes_list), binary=False)